## Feature selection

* Select the best features available;
* Don't select unecessary features;
* Create new features

## Adding a new feature

1. human intuition;
2. code up the new feature;
3. visualize;
4. repeat.

## Getting rid of features

* It's noisy;
* It causes overfitting;
* It is strongly related (highly correlated) with a feature that's already present;
* Addtional features slow down training/testing process

## Features != Information

* We want information to draw conclusions and have insights;
* Features is the actual number or caracteristics of a particular data point that's attempting to access information;
* Is a little bit like the difference between the quantity of something and the quality;
* Example of prepross is in `tools/email_preprocess.py`

## Sklearn options

* `SelectPercentile`: Selects the % of strong features;
* `SelectKBest`: Select k strong features.
* In `TfidfVectorizer` the argument `max_df` can be used for feature reduction (dimensionality reduction)